In [1]:
import chess, os, chess.pgn
import numpy as np
import tensorflow as tf
import glob

from model import ResNet, create_model_v2
import utils
conf = utils.Config()

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2060, compute capability 7.5


2022-07-29 12:45:47.072332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 12:45:47.109552: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 12:45:47.109809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 12:45:47.110461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
def gen(path=None):
    planes = None
    output_array = np.zeros([*conf.BOARD_SHAPE, conf.N_PLANES], dtype=np.float32)
    
    if path == None:
        database_path = '/home/marcello/github/ChessBreaker/data/Database'
        files = glob.glob(os.path.join(database_path, '*.pgn'))
    else:
        path = path.decode("utf-8")
        files = [path]
    
    for filename in files:
        with open(os.path.join(os.getcwd(), filename), 'r') as pgn:
            game = chess.pgn.read_game(pgn)

            while game != None:
                whole_game_moves = game.game().mainline_moves()
                
                result = utils.outcome(game.headers["Result"])
                if result == None:
                    result = 0

                board = chess.Board()
                board_history = [board.fen()[:-6]]
                
                for move in whole_game_moves:
                    # the input is the PREVIOUS board
                    planes = utils.update_planes(planes, board, board_history)
                    # inputs.append(planes)
                    
                    # the output is the move from that position
                    mask = utils.mask_moves([move])[0]
                    output_array[mask[0], mask[1], mask[2]] = 1
                    # outputs.append(output_array)

                    # oss: input = planes, output = (moves + result)!!
                    yield (planes, (output_array, result)) ### yield before resetting the output

                    output_array[mask[0], mask[1], mask[2]] = 0
                    
                    # then you actually push the move preparing for next turn
                    board.push(move)
                    board_history.append(board.fen()[:-6])
                
                game = chess.pgn.read_game(pgn)

# avg = np.average([info[file, "games"] for file in info.keys()])

In [3]:
# train_file_path = "/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-11.pgn"  # 19547 samples

val_file_path = "/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2015-12.pgn"    # 446'402 samples

output_signature=(
    tf.TensorSpec((8,8,119), dtype=tf.dtypes.float16),
    (
        tf.TensorSpec((8,8,73), dtype=tf.dtypes.float32),
        tf.TensorSpec((1,), dtype=tf.dtypes.float32)
    )
)

dataset = tf.data.Dataset.from_generator(gen, output_signature=output_signature)                            # training is 313'831'972 moves

val_dataset = tf.data.Dataset.from_generator(gen, output_signature=output_signature, args=[val_file_path])  # validation is 446'402 moves --> we don't want to loose to much time

# oss: total training is 5,4 milion total steps, so 150k shuffle buffer should be enough 
# (also because a match is ~200 steps, you want just to remove the correlation of moves inside games)
ds = dataset.shuffle(conf.BATCH_DIM*10) \
    .batch(2, num_parallel_calls=tf.data.AUTOTUNE) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = val_dataset.batch(conf.IMITATION_LEARNING_BATCH, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

2022-07-29 12:45:47.184588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-29 12:45:47.185470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 12:45:47.185691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 12:45:47.185811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
# inputs = tf.keras.Input(shape=(8,8,119))
# x =         tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu")(inputs)
# output1 =   tf.keras.layers.Conv2D(73,  3, padding="same", activation="relu")(x)
# output2 =   tf.keras.layers.Dense(1,       activation="relu")(x)
# model = tf.keras.Model(inputs=inputs, outputs=[output1, output2], name="try1")

In [5]:
# model = ResNet()
model = create_model_v2()


optimizer = tf.keras.optimizers.Adam()

losses = {
	"policy": tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
	"value": tf.keras.losses.MeanSquaredError(),
}
model.compile(
    optimizer = optimizer,
    loss = losses
)

In [6]:
model(tf.stack([tf.zeros([8, 8, 119])]*8, axis = 0))

2022-07-29 12:45:48.759434: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302


[<tf.Tensor: shape=(8, 8, 8, 73), dtype=float16, numpy=
 array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 8, 119)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 8, 8, 256)    274432      ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 8, 8, 256)   1024        ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 8, 8, 256)    0           ['batch_normalization[0][0]']

In [7]:
# tf.keras.utils.plot_model(model, "mini_resnet.png", show_shapes=True)

In [8]:
# sam = 0
for sample in ds.take(1):
    planes, (move, outcome) = sample
    print(np.shape(move))
    print(utils.custom_accuracy(move, model(planes)[0]))
#     print(np.shape(planes))
#     print(np.shape(move))
#     print(move.dtype)
#     print(np.shape(outcome))
#     print(move.dtype)

#     move, outcome = model(planes)
#     print(np.shape(move))
#     print(move.dtype)
#     print(np.shape(outcome))
#     print(move.dtype)


(2, 8, 8, 73)


2022-07-29 11:10:18.616049: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302


[6 0]


In [9]:
# callbacks = [
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_lr=1e-5),
#     tf.keras.callbacks.ModelCheckpoint(filepath="tmp/checkpoint", monitor='val_loss', save_freq='epoch'),
#     tf.keras.callbacks.TensorBoard(log_dir='logs'),
#     tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12)
# ]

# history = model.fit(
#     ds,
#     validation_data = val_ds,
#     epochs = 60,
#     steps_per_epoch = 10000,    # 10000*60*512
#     callbacks = callbacks,
#     workers = 16,
#     use_multiprocessing = True
# )


In [15]:
database_path = '/home/marcello/github/ChessBreaker/data/Database'
files = glob.glob(os.path.join(database_path, '*.pgn'))
files.sort()

In [17]:
print(files)

['/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2013-09.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2013-11.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-01.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-02.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-03.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-04.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-05.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-06.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-07.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-08.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-09.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-10.pgn', '/home/marcello/github/ChessBreaker/data/Database/lichess_elite